<a href="https://colab.research.google.com/github/sugarforever/LangChain-Tutorials/blob/main/LangChain_LLM_Math.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 666.0/666.0 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 6.3 MB/s eta 0:00:00


In [23]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

In [3]:
OPENAI_API_KEY = 'your OpenAI API key here'

In [24]:
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, temperature=0, model_name="gpt-3.5-turbo")

In [25]:
from langchain.tools import BaseTool

class EvaluateMathExpression(BaseTool):
    name = "Math Evaluation"
    description = 'use this tool to evaluate a math expression.'

    def _run(self, expr: str):
        return eval(expr)
    
    def _arun(self, query: str):
        raise NotImplementedError("Async operation not supported yet")

tools = [EvaluateMathExpression()]

In [27]:
agent = initialize_agent(
    agent='chat-conversational-react-description',
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3,
    early_stopping_method='generate',
    memory=ConversationBufferWindowMemory(
        memory_key='chat_history',
        k=5,
        return_messages=True
    )
)

In [28]:
for message in agent.agent.llm_chain.prompt.messages:
  print(message)

prompt=PromptTemplate(input_variables=[], output_parser=None, partial_variables={}, template='Assistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nAssistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide

In [29]:
agent(f"What is 2 * 2 * 0.13 - 1.001?")



> Entering new AgentExecutor chain...
{
    "action": "Final Answer",
    "action_input": "-0.341"
}

> Finished chain.


{'input': 'What is 2 * 2 * 0.13 - 1.001?',
 'chat_history': [],
 'output': '-0.341'}

In [30]:
2 * 2 * 0.13 - 1.001

-0.48099999999999987

In [31]:
from langchain.agents.conversational_chat.prompt import (PREFIX)
system_message = PREFIX + "\n" + '''
Unfortunately, Assistant is terrible at maths. Assistant should always refers to available tools and never try to answer math questions by itself
'''

In [32]:
new_prompt = agent.agent.create_prompt(
    system_message=system_message,
    tools=tools
)

agent.agent.llm_chain.prompt = new_prompt

In [33]:
agent(f"What is 2 * 2 * 0.13 - 1.001?")



> Entering new AgentExecutor chain...
{
    "action": "Math Evaluation",
    "action_input": "2 * 2 * 0.13 - 1.001"
}
Observation: -0.48099999999999987
Thought:{
    "action": "Final Answer",
    "action_input": "-0.481"
}

> Finished chain.


{'input': 'What is 2 * 2 * 0.13 - 1.001?',
 'chat_history': [HumanMessage(content='What is 2 * 2 * 0.13 - 1.001?', additional_kwargs={}),
  AIMessage(content='-0.341', additional_kwargs={})],
 'output': '-0.481'}

In [34]:
agent.memory.buffer

[HumanMessage(content='What is 2 * 2 * 0.13 - 1.001?', additional_kwargs={}),
 AIMessage(content='-0.341', additional_kwargs={}),
 HumanMessage(content='What is 2 * 2 * 0.13 - 1.001?', additional_kwargs={}),
 AIMessage(content='-0.481', additional_kwargs={})]